In [ ]:
!pip install google-cloud-bigquery

In [ ]:
!pip install google_auth

In [ ]:
import pandas as pd
import numpy as np
import datetime
from google.cloud import bigquery
from datetime import datetime

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving inductive-time-410214-f7a634500b87.json to inductive-time-410214-f7a634500b87.json


In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/inductive-time-410214-f7a634500b87.json"

In [ ]:
queries = [
    "select sum(lo_extendedprice*lo_discount) as revenue from inductive-time-410214.sf10.lineorder_obt where d_order_yearmonthnum = 199401 and lo_discount between 4 and 6 and lo_quantity between 26 and 35;",

    "select sum(lo_extendedprice*lo_discount) as revenue from inductive-time-410214.sf10.lineorder_obt where d_order_year = 1993 and lo_discount between 1 and 3 and lo_quantity < 25;",
    "select sum(lo_extendedprice*lo_discount) as revenue from inductive-time-410214.sf10.lineorder_obt where d_order_weeknuminyear = 6 and d_order_year = 1994 and lo_discount between 5 and 7 and lo_quantity between 26 and 35;",
    "select sum(lo_revenue), d_order_year, p_brand from inductive-time-410214.sf10.lineorder_obt where p_category = 'MFGR#12' and s_region = 'AMERICA' group by d_order_year, p_brand order by d_order_year, p_brand;",
    "select sum(lo_revenue), d_order_year, p_brand from inductive-time-410214.sf10.lineorder_obt where p_brand between 'MFGR#2221' and 'MFGR#2228' and s_region = 'ASIA' group by d_order_year, p_brand order by d_order_year, p_brand;",
    "select sum(lo_revenue), d_order_year, p_brand from inductive-time-410214.sf10.lineorder_obt where p_brand = 'MFGR#2221' and s_region = 'EUROPE' group by d_order_year, p_brand order by d_order_year, p_brand;",
    "select c_nation, s_nation, d_order_year, sum(lo_revenue) as revenue from inductive-time-410214.sf10.lineorder_obt where c_region = 'ASIA' and s_region = 'ASIA' and d_order_year >= 1992 and d_order_year <= 1997 group by c_nation, s_nation, d_order_year order by d_order_year asc, revenue desc;",
    "select c_city, s_city, d_order_year, sum(lo_revenue) as revenue from inductive-time-410214.sf10.lineorder_obt where c_nation = 'UNITED STATES' and s_nation = 'UNITED STATES' and d_order_year >= 1992 and d_order_year <= 1997 group by c_city, s_city, d_order_year order by d_order_year asc, revenue desc;",
    "select c_city, s_city, d_order_year, sum(lo_revenue) as revenue from inductive-time-410214.sf10.lineorder_obt where d_order_year >= 1992 and (c_city='UNITED KI1' or c_city='UNITED KI5') and (s_city='UNITED KI1' or s_city='UNITED KI5') and d_order_year <= 1997 group by c_city, s_city, d_order_year order by d_order_year asc, revenue desc;",
    "select c_city, s_city, d_order_year, sum(lo_revenue) as revenue from inductive-time-410214.sf10.lineorder_obt where d_order_yearmonth = 'Dec1997' and (c_city='UNITED KI1' or c_city='UNITED KI5') and (s_city='UNITED KI1' or s_city='UNITED KI5') group by c_city, s_city, d_order_year order by d_order_year asc, revenue desc;",
    "select d_order_year, c_nation, sum(lo_revenue - lo_supplycost) as profit from inductive-time-410214.sf10.lineorder_obt where c_region = 'AMERICA'  and s_region = 'AMERICA'  and (p_mfgr = 'MFGR#1' or p_mfgr = 'MFGR#2') group by d_order_year, c_nation order by d_order_year, c_nation;",
    "select d_order_year, s_nation, p_category, sum(lo_revenue - lo_supplycost) as profit from inductive-time-410214.sf10.lineorder_obt where c_region = 'AMERICA' and s_region = 'AMERICA' and (d_order_year = 1997 or d_order_year = 1998) and (p_mfgr = 'MFGR#1' or p_mfgr = 'MFGR#2') group by d_order_year, s_nation, p_category order by d_order_year, s_nation, p_category;",
    "select d_order_year, s_city, p_brand, sum(lo_revenue - lo_supplycost) as profit from inductive-time-410214.sf10.lineorder_obt where c_region = 'AMERICA' and s_nation = 'UNITED STATES' and (d_order_year = 1997 or d_order_year = 1998) and p_category = 'MFGR#14' group by d_order_year, s_city, p_brand order by d_order_year, s_city, p_brand;"
]

In [ ]:
queries =  [
        "select sum((lo_extendedprice*lo_discount)) as revenue from sf10.lineorder, sf10.dwdate where d_yearmonthnum = 199401 and lo_orderdate = d_datekey and lo_discount between 4 and 6 and lo_quantity between 26 and 35;",
        "select sum((lo_extendedprice*lo_discount)) as revenue from sf10.lineorder, sf10.dwdate where d_year = 1993 and lo_orderdate = d_datekey and lo_discount between 1 and 3 and lo_quantity < 25;",
        "select sum((lo_extendedprice*lo_discount)) as revenue from sf10.lineorder, sf10.dwdate where d_weeknuminyear = 6 and lo_orderdate = d_datekey and d_year = 1994 and lo_discount between 5 and 7 and lo_quantity between 26 and 35;",
        "select sum((lo_revenue)), d_year, p_brand from sf10.lineorder, sf10.dwdate, sf10.part, sf10.supplier where p_category = 'MFGR#12' and lo_orderdate = d_datekey and lo_partkey = p_partkey and lo_suppkey = s_suppkey and s_region = 'AMERICA' group by d_year, p_brand order by d_year, p_brand;",
        "select sum((lo_revenue)), d_year, p_brand from sf10.lineorder, sf10.dwdate, sf10.part, sf10.supplier where p_brand BETWEEN 'MFGR#2221' and  'MFGR#2228' and s_region = 'ASIA' and lo_orderdate = d_datekey and lo_partkey = p_partkey and lo_suppkey = s_suppkey group by d_year, p_brand order by d_year, p_brand;",
        "select sum((lo_revenue)), d_year, p_brand from sf10.lineorder, sf10.dwdate, sf10.part, sf10.supplier where p_brand = 'MFGR#2221' and lo_orderdate = d_datekey and lo_partkey = p_partkey and lo_suppkey = s_suppkey and s_region = 'EUROPE' group by d_year, p_brand order by d_year, p_brand;",
        "select c_nation, s_nation, d_year, sum((lo_revenue)) as revenue from sf10.customer, sf10.lineorder, sf10.supplier, sf10.dwdate where lo_custkey = c_custkey and lo_suppkey = s_suppkey and lo_orderdate = d_datekey and c_region = 'ASIA' and s_region = 'ASIA' and d_year >= 1992 and d_year <= 1997 group by c_nation, s_nation, d_year order by d_year asc, revenue desc;",
        "select c_city, s_city, d_year, sum((lo_revenue)) as revenue from sf10.customer, sf10.lineorder, sf10.supplier, sf10.dwdate where c_nation = 'UNITED STATES' and lo_custkey = c_custkey and lo_suppkey = s_suppkey and lo_orderdate = d_datekey and s_nation = 'UNITED STATES' and d_year >= 1992 and d_year <= 1997 group by c_city, s_city, d_year order by d_year asc, revenue desc;",
        "select c_city, s_city, d_year, sum((lo_revenue)) as revenue from sf10.customer, sf10.lineorder, sf10.supplier, sf10.dwdate where d_year >= 1992 and lo_custkey = c_custkey and lo_suppkey = s_suppkey and lo_orderdate = d_datekey and (c_city='UNITED KI1' or c_city='UNITED KI5') and (s_city='UNITED KI1' or s_city='UNITED KI5') and d_year <= 1997 group by c_city, s_city, d_year order by d_year asc, revenue desc;",
        "select c_city, s_city, d_year, sum((lo_revenue)) as revenue from sf10.customer, sf10.lineorder, sf10.supplier, sf10.dwdate where d_yearmonth = 'Dec1997' and lo_custkey = c_custkey and lo_suppkey = s_suppkey and lo_orderdate = d_datekey and (c_city='UNITED KI1' or c_city='UNITED KI5') and (s_city='UNITED KI1' or s_city='UNITED KI5') group by c_city, s_city, d_year order by d_year asc, revenue desc;",
        "select d_year, c_nation, sum((lo_revenue - lo_supplycost)) as profit from sf10.dwdate, sf10.customer, sf10.supplier, sf10.part, sf10.lineorder where lo_custkey = c_custkey  and lo_suppkey = s_suppkey  and lo_partkey = p_partkey  and lo_orderdate = d_datekey  and c_region = 'AMERICA'  and s_region = 'AMERICA'  and (p_mfgr = 'MFGR#1' or p_mfgr = 'MFGR#2') group by d_year, c_nation order by d_year, c_nation;",
        "select d_year, s_nation, p_category, sum((lo_revenue - lo_supplycost)) as profit from sf10.dwdate, sf10.customer, sf10.supplier, sf10.part, sf10.lineorder where lo_custkey = c_custkey and lo_suppkey = s_suppkey and lo_partkey = p_partkey and lo_orderdate = d_datekey and c_region = 'AMERICA' and s_region = 'AMERICA' and (d_year = 1997 or d_year = 1998) and (p_mfgr = 'MFGR#1' or p_mfgr = 'MFGR#2') group by d_year, s_nation, p_category order by d_year, s_nation, p_category;",
        "select d_year, s_city, p_brand, sum((lo_revenue - lo_supplycost)) as profit from sf10.dwdate, sf10.customer, sf10.supplier, sf10.part, sf10.lineorder where lo_custkey = c_custkey and lo_suppkey = s_suppkey and lo_partkey = p_partkey and lo_orderdate = d_datekey and c_region = 'AMERICA' and s_nation = 'UNITED STATES' and (d_year = 1997 or d_year = 1998) and p_category = 'MFGR#14' group by d_year, s_city, p_brand order by d_year, s_city, p_brand;"
        ]

In [ ]:
client = bigquery.Client()

for i in range(len(queries)):
  arr_elapsed_time = []
  arr_bytes_processados = []
  arr_total_slots = []
  for _ in range(1,31):
    # Sua consulta original
    start_time = datetime.utcnow()
    sua_consulta = queries[i]

    # Configuração para evitar o cache
    configuracao_query = bigquery.QueryJobConfig(use_query_cache=False)

    # Execute a consulta com a configuração
    query_job = client.query(sua_consulta, job_config=configuracao_query)

    end_time = datetime.utcnow()

    job_id = query_job.job_id

    # Obtenha detalhes do job
    job_details = client.get_job(job_id)
    #total_time = job_details.ended - job_details.created
    #time_d_ms  = total_time / datetime.timedelta(milliseconds=1)
    elapsed_time = (end_time - start_time).total_seconds()


    bytes_processados = job_details.total_bytes_processed

    total_slot_ms = job_details.slot_millis

    arr_elapsed_time.append(elapsed_time*1000)
    arr_bytes_processados.append(bytes_processados)

    arr_total_slots.append(total_slot_ms)
    #print("Elapsed Time:", total_time)
    #print("Bytes Processados:", job_details.total_bytes_processed)


    df = pd.DataFrame()

    lista_de_tuplas = list(zip(arr_elapsed_time, arr_bytes_processados,arr_total_slots))
    df = pd.DataFrame(
      lista_de_tuplas,
      columns=['elapsed_time_ms', 'bytes_processados', 'total_slot_ms']
    )

    df.to_csv('query_sf10_lineorder_obt_'+str(i)+'.csv')



In [ ]:




from google.cloud import bigquery

# Criar um cliente BigQuery
cliente = bigquery.Client()

consulta = "select sum((lo_extendedprice*lo_discount)) as revenue from sf100.lineorder, sf100.dwdate where d_yearmonthnum = 199401 and lo_orderdate = d_datekey and lo_discount between 4 and 6 and lo_quantity between 26 and 35;"


query_job = cliente.query(consulta)

# Substituir 'seu_job_id' pelo ID real do seu job
job_id = query_job.job_id

# Obter o objeto Job usando o job_id
job_details = cliente.get_job(job_id)



print(job_details.slot_millis)

print(job_details.script_statistics)





None
None


In [ ]:
from google.cloud import bigquery

# Criar um cliente BigQuery
cliente = bigquery.Client()

# Executar uma consulta
consulta = "select sum((lo_extendedprice*lo_discount)) as revenue from sf100.lineorder, sf100.dwdate where d_yearmonthnum = 199401 and lo_orderdate = d_datekey and lo_discount between 4 and 6 and lo_quantity between 26 and 35;"

job = cliente.query(consulta)

# Aguardar a conclusão da consulta
resultado = job.result()

# Verificar se o job foi concluído
if job.done():
    # Acessar as estatísticas da consulta
    estatisticas = resultado._query_results().statistics

    # Exibir informações de desempenho
    print(f"Total de slots de milissegundos: {estatisticas['totalSlotMs']}")

    # Agora você pode iterar sobre os resultados usando o RowIterator
    for linha in resultado:
        print(linha)
else:
    print("O job ainda não foi concluído.")



AttributeError: 'RowIterator' object has no attribute '_query_results'